## naver의 영화리뷰 데이터를 이용한 감성 분류 모델 구축

In [ ]:
import pandas as pd
import numpy as np
import warnings
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import json
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [7]:
warnings.filterwarnings(action= 'ignore')

### 1. 데이터 준비

In [8]:
rating_tr = pd.read_csv("C:/Users/kimjunseok/Downloads/practice_note/data/ratings_train.txt", encoding= 'utf8', sep='\t', engine = 'python')
rating_tr.head()
#label 클래스의 0은 부정 감성, 1은 긍정 감성

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
rating_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [10]:
#document에 존재하는 결측치 제거
rating_tr = rating_tr.dropna(how='any') #1개라도 결측치가 존재하는 행은 제거
len(rating_tr)

149995

In [11]:
#감성 분류 클래스의 구성 확인
rating_tr.label.value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [12]:
#한글 이외의 문자는 공백으로 변환

rating_tr['document'] =rating_tr['document'].apply(lambda x : re.sub(r'[^ ㄱ - |가-힣]+', " ", x))
rating_tr.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [14]:
#test data에도 동일한 처리
rating_te = pd.read_csv("C:/Users/kimjunseok/Downloads/practice_note/data/ratings_test.txt", encoding= 'utf8', sep='\t', engine = 'python')
rating_te.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


### 2. 피처 백터화: TF-IDF
#### (1) 형태소를 분석하여 토큰화: 한글 형태소 엔진으로 Okt 이용 

In [15]:
okt = Okt()

In [16]:
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

#### (2) TF-IDF 기반 피처 벡터 생성

In [17]:
tfidf = TfidfVectorizer(tokenizer= okt_tokenizer, ngram_range=(1,2), min_df= 3, max_df=0.9)
tfidf.fit(rating_tr['document'])
rating_tr_tfidf = tfidf.transform(rating_tr['document'])

## 3. Sentiment Analysis using Logistic Regression

### (1) 로지스틱 회귀 기반 분석모델 생성

In [ ]:
SA_lr = LogisticRegression(random_state = 0)

In [ ]:
SA_lr.fit(rating_tr_tfidf, rating_tr['label'])

### (2) 로지스틱 회귀의 best 하이퍼 파라미터 찾기

In [ ]:
params = {'C': [1,3,3.5, 4,4.5, 5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid= params, cv = 3, scoring ='accuracy', verbose=1)

### (3) 최적 분석 모델 훈련

In [ ]:
SA_lr_grid_cv.fit(rating_tr_tfidf, rating_tr['label'])

In [ ]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

In [ ]:
#최적 파라미터의 best 모델 저장
SA_lr_best = SA_lr.best_estimator_

### 4. 분석 모델 평가
#### (1) 평가용 데이터를 이용하여 감성 분석 모델 정확도

In [ ]:
#평가용 데이터의 피처 벡터화
rating_tr_tfidf = tfidf.transform(rating_te['document'])

In [ ]:
test_predict = SA_lr_best.predict(nsmc_test_tfidf)

In [ ]:
print('감성 분석 정확도 : ', round(accuracy_score(nsmc_test_df['label'], test_predict), 3))

#### (2) 새로운 텍스트에 대한 감성 예측

In [ ]:
st = input('감성 분석할 문장 입력 >> ')

In [ ]:
# 0)  입력 테스트에 대한 전처리 수행
st = re.compile(r'[ㄱ-|-힣]+').findall(st)
print(st)
st = [" ".join(st)]
print(st)

In [ ]:
# 1) 입력 테스트의 피처 벡터화
st_tfidf = tfidf.transform(st)

# 2) 최적 감성분석 모델에 적용하여 감성 분석 평가
st_predict = SA_lr_best.predict(st_tfidf)

In [ ]:
# 3)예측 값 출력하기
if (st_predict == 0):
    print(st, "->> 부정 감성") 
else :
    print(st, "->> 긍정 감성")

## Naver에서 크롤링한 뉴스 데이터로 감성 분석

### 1. 데이터 준비

In [ ]:
with open("C:/Users/kimjunseok/Downloads/practice_note/data/news.json", encoding = 'utf8') as j_f:
    data = json.load(j_f)

In [ ]:
print(data)

In [ ]:
#분석할 컬럼을 추출하여 데이터 프레임에 저장
data_title = []
data_description =  []

for item in data:
    data_title.append(item['title'])
    data_description.append(item['description'])

In [ ]:
data_title

In [ ]:
data_description

In [ ]:
data_df = pd.DataFrame({'title': data_title , 'description' : data_description})

In [ ]:
#한글 이외 문자 제거
data_df['title'] =data_df['title'].apply(lambda x : re.sub(r'[^ ㄱ - |가-힣]+', " ", x))
data_df['description'] = data_df['description'].apply(lambda x : re.sub(r'[^ ㄱ - |가-힣]+', " ", x))

data_df.head()  #w작업 확인 용 출력

### 2. 감성 분석 수행
#### (1) 'title'에 대한 감성 분석

In [ ]:
# 1) 분석할 데이터의 피처 벡터화 ---<< title >> 분석
data_title_tfidf = tfidf.transform(data_df['title'])

# 2) 최적 파라미터 학습 모델에 적용하여 감성 분석
data_title_predict = SA_lr_best.predict(data_title_tfidf)

# 3) 감성 분석 결과값을 데이터 프레임에 저장
data_df['title_label'] = data_title_predict

#### (2) 'description'에 대한 감성 분석

In [ ]:
# 1) 분석할 데이터의 피처 벡터화 ---<< title >> 분석
data_description_tfidf = tfidf.transform(data_df['description'])

# 2) 최적 파라미터 학습 모델에 적용하여 감성 분석
data_description_predict = SA_lr_best.predict(data_description_tfidf)

# 3) 감성 분석 결과값을 데이터 프레임에 저장
data_df['description_label'] = data_description_predict

### 3. 감성 분석 결과 확인 - 0: 부정 감성, 1: 긍정감성

In [ ]:
#감성 분석 결과 확인 
data_df.head()

In [ ]:
print(data_df['description_label'].value_counts())

In [ ]:
columns_name = ['title', 'title_label', 'description', 'description_label']
NEG_data_df = pd.DataFrame(columns=columns_name)
POS_data_df = pd.DataFrame(columns=columns_name)

for i, data in data_df.iterrows():
    title = data['title']
    description = data['description']
    t_label = data["title_label"]
    d_label = data["description_label"]
    
    if d_label == 0: # 부정 감성 샘플만 추출
        NEG_data_df = NEG_data_df.append(pd.DataFrame([[title, t_label , description , d_label]], columns = columns_name))
        
    else: # 긍정 감성 샘플만 추출
        POS_data_df = POS_data_df.append(pd.DataFrame([[title, t_label , description , d_label]], columns = columns_name))

In [ ]:
len(NEG_data_df) , len(POS_data_df)

### 4. 감성 분석 결과 시각화: 막대 도표
#### (1) 명사만 추출하여 정리하기

In [ ]:
# 긍정 감성의 데이터에서 명사만 추출하여 정리
POS_description = POS_data_df['description']
POS_description_noun_tk = []

for d in POS_description:
    POS_description_noun_tk.append(okt.nouns(d))  #형태소가 명사인 것만 추출
    
print(POS_description_noun_tk) #작업 확인용 출력

In [ ]:
POS_description_noun_join = []

for d in POS_description_noun_tk:
    d2 = [w for w in d if len(w) > 1] #길이가 1인 토큰은 제외
    POS_description_noun_join.append(" ".join(d2)) #토큰을 연결(join)햐여 리스트 구성

In [ ]:
print(POS_description_noun_join) #작업 확인용 출력

In [ ]:
# 부정 감성의 데이터에서 명사만 추출하여 정리

NEG_description =NEG_data_df['description']

NEG_description_noun_tk = []
NEG_description_noun_join = []

for d in NEG_description:
    NEG_description_noun_tk.append(okt.nouns(d))  #형태소가 명사인 것민 추출
    
for d in NEG_description_noun_tk:
    d2 = [ w for w in d if len(w) > 1] #길이가 1인 토큰은 제외
    NEG_description_noun_join.append(" ".join(d2))  #토큰을 연결(join)하여 리스트 구성
    
print(NEG_description_noun_join)

#### (2) DTM 구축

In [ ]:
#긍정 감성 데이터에 대한 dtm 구성, dtm을 이용하여 단어 사전 구성 후 내림차순 구성 후 내림차순 정렬
POS_tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, min_df =2)
POS_dtm = POS_tfidf.fit_transform(POS_description_noun_join)
POS_vocab = dict()

for idx , word in enumerate(POS_tfidf.get_feature_names()):
    POS_vocab[word] = POS_dtm.getcol(idx).sum()
    
POS_words = sorted(POS_vocab.items(), key=lambda x:x[1], reverse = True)
POS_words # 작업확인용 출력

In [ ]:
#부정 감성 데이터에 대한 dtm 구성, dtm을 이용하여 단어 사전 구성 후 내림차순 구성 후 내림차순 정렬
NEG_tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, min_df =2)
NEG_dtm = NEG_tfidf.fit_transform(NEG_description_noun_join)
NEG_vocab = dict()

for idx , word in enumerate(NEG_tfidf.get_feature_names()):
    NEG_vocab[word] = NEG_dtm.getcol(idx).sum()
    
NEG_words = sorted(NEG_vocab.items(), key=lambda x:x[1], reverse = True)
NEG_words # 작업확인용 출력

#### (3) 단어 사전의 상위 단어로 바 차트 그리기

In [ ]:
fm.get_fontconfig_fonts()
font_location = "C:/Windows/Fonts/malgun.ttf"
font_name = fm.FontProperties(fname= font_location).get_nmae()
matplotlib.rc('font', family = font_name)

max = 15 #바 차트에 나타낼 단어의 수


In [ ]:
plt.bar(range(max), [i[1] for i in POS_words[:max]], color = "blue")
plt.title("긍정 뉴스의 단어 상위 %개" %max, fontsize = 15)
plt.xlabel("단어", fontsize =12)
plt.ylabel("TF-IDF의 합", fontsize = 12)
plt.xticks(range(max), [i[0] for i in POS_words[:max]], rotation = 70)

plt.show()

In [ ]:
plt.bar(range(max), [i[1] for i in NEG_words[:max]], color = "red")
plt.title("부정 뉴스의 단어 상위 %개" %max, fontsize = 15)
plt.xlabel("단어", fontsize =12)
plt.ylabel("TF-IDF의 합", fontsize = 12)
plt.xticks(range(max), [i[0] for i in NEG_words[:max]], rotation = 70)

plt.show()